In [29]:
import math
import random


In [30]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0

        self._backward = lambda: None#we set this according to whatever op is performed
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __radd__(self, other):#due to one error that occured
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
         return self*-1

    def __sub__(self, other):
        return self + (-other)

    def __pow__(self, power):
        out = Value(self.data ** power, (self,), f'**{power}')

        def _backward():
            self.grad += power * (self.data ** (power - 1)) * out.grad
        out._backward = _backward

        return out

    def tanh(self):#loss function im using
        t = (math.exp(2*self.data) - 1) / (math.exp(2*self.data) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward

        return out

    def backward(self):#the main algo
        topo = []
        visited = set()

        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)

        build(self)
        self.grad = 1.0

        for v in reversed(topo):
            v._backward()


In [31]:
class Neuron:#neuron for the implementation
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0.0)

    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

    def parameters(self):
        return self.w + [self.b]


In [32]:
class Layer:#collection of neurons
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]


In [33]:
class MLP:#the model
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [34]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]#the model example

ys = [1.0, -1.0, -1.0, 1.0]

model = MLP(3, [4, 4, 1])


In [35]:
for epoch in range(50):

    ypred = [model(x) for x in xs]
    loss = sum((yout - ygt)**2 for yout, ygt in zip(ypred, ys))#loss function

    for p in model.parameters():
        p.grad = 0.0

    loss.backward()#calculating the loss

    for p in model.parameters():
        p.data += -0.05 * p.grad

    print(f"Epoch {epoch}, Loss = {loss.data}")


Epoch 0, Loss = 7.385569862407149
Epoch 1, Loss = 4.558917325500576
Epoch 2, Loss = 3.20151016640809
Epoch 3, Loss = 2.190421703020123
Epoch 4, Loss = 1.3688543144047736
Epoch 5, Loss = 0.7956708470058451
Epoch 6, Loss = 0.4796490801585076
Epoch 7, Loss = 0.32186182615338527
Epoch 8, Loss = 0.23709926661523856
Epoch 9, Loss = 0.1851871715823451
Epoch 10, Loss = 0.15061659234794772
Epoch 11, Loss = 0.1261916838978834
Epoch 12, Loss = 0.10813916486497932
Epoch 13, Loss = 0.09431863691080095
Epoch 14, Loss = 0.08343640628992235
Epoch 15, Loss = 0.07466919249866848
Epoch 16, Loss = 0.06747039615424343
Epoch 17, Loss = 0.06146405962168295
Epoch 18, Loss = 0.056383755907089675
Epoch 19, Loss = 0.052035794412357034
Epoch 20, Loss = 0.04827622077181689
Epoch 21, Loss = 0.04499596509302614
Epoch 22, Loss = 0.04211097622590591
Epoch 23, Loss = 0.03955550271072762
Epoch 24, Loss = 0.0372774145196784
Epoch 25, Loss = 0.03523488087040656
Epoch 26, Loss = 0.03339396890285741
Epoch 27, Loss = 0.03172